---

**Runtime Configuration:** This notebook has a paired setup script at `runtimes/fly_connectome_02_neuron_morphology_post_startup.sh` which provides the complete installation recipe for all dependencies. This script can be used as a post-startup script for Google Colab to automatically configure the environment.

---

# Core Tutorial

## Introduction

The purpose of this tutorial is to examine neuron morphology.

We can visualise neurons, compare their morphological similarity with [NBLAST](https://github.com/navis-org/navis), and plot them with their synapses.

To do this, we are going to look at one of the data 'subsets' we have pre-prepared.

I have subset the synapses and edgelist relevant for a few interesting areas, so let's use one of those.

You can change which subset and dataset to examine by changing the variables in the setup cells below.

By default, let us look at the **front leg neuropil**!

This is available for BANC, maleCNS and MANC.


## Setup and Configuration

## Import Packages

We'll use:
- **navis**: Neuron analysis and visualization
- **pandas**: Data manipulation
- **pyarrow**: Reading Feather files
- **gcsfs**: Google Cloud Storage access
- **plotly**: Interactive 3D visualizations
- **trimesh**: 3D mesh processing

**Installation:**
```bash
pip install navis[all] flybrains gcsfs plotly kaleido trimesh
```

In [1]:
# Environment detection and Colab setup (auto-configured)
try:
    import google.colab
    IN_COLAB = True
    
    # Colab setup
    
    # Authenticate
    from google.colab import auth
    auth.authenticate_user()
    print("✓ Authenticated with Google Cloud")
    
    # Download utils.py
    import urllib.request, os
    HELPER_URL = "https://raw.githubusercontent.com/sjcabs/fly_connectome_data_tutorial/main/python/utils.py"
    if not os.path.exists("utils.py"):
        urllib.request.urlretrieve(HELPER_URL, "setup_helpers.py")
    
    print("✓ Colab environment ready\n")
except ImportError:
    IN_COLAB = False
    # Local environment - no output needed
    pass

In [2]:
# Re-declare configuration for notebook execution
DATASET = "banc_746"
DATASET_ID = "banc_746_id"
SUBSET_NAME = "front_leg"
NEUROPIL_PATTERN = "T1"
DATA_PATH = "gs://sjcabs_2025_data"
USE_GCS = DATA_PATH.startswith("gs://")
import os
IMG_DIR = "images/tutorial_02"
os.makedirs(IMG_DIR, exist_ok=True)


# Import all common packages and helper functions
import sys
sys.path.insert(0, '.')
from utils import *

print(f"navis version: {navis.__version__}")
print(f"pandas version: {pd.__version__}")

✓ Packages loaded successfully
navis version: 1.10.0
pandas version: 2.3.3


## Setup GCS Access

**Authentication required:** Before running with GCS, authenticate:

```bash
gcloud auth application-default login
```

In [3]:
# Setup GCS filesystem if needed
if USE_GCS:
    gcs = gcsfs.GCSFileSystem(token='google_default')
    print("✓ GCS filesystem initialized")
else:
    gcs = None
    print("Using local filesystem")

✓ GCS filesystem initialized


## Setup File Paths

In [4]:
# Construct paths
meta_path = construct_path(DATA_PATH, DATASET, "meta")
skeletons_path = construct_path(DATA_PATH, DATASET, "skeletons")

# Extract base dataset name
dataset_base = DATASET.split("_")[0]

print("File paths:")
print(f"  Metadata: {meta_path}")
print(f"  Skeletons: {skeletons_path}")

File paths:
  Metadata: gs://sjcabs_2025_data/banc/banc_746_meta.feather
  Skeletons: gs://sjcabs_2025_data/banc/banc_banc_space_l2_swc


### Load Subset Data

We'll load the subset edgelist to identify which neurons are in the front leg subset, then filter the metadata accordingly.

In [5]:
# Construct path to subset edgelist
dataset_base = DATASET.split('_')[0]
subset_dir = f"{DATA_PATH}/{dataset_base}/{SUBSET_NAME}"
edgelist_path = f"{subset_dir}/{DATASET}_{SUBSET_NAME}_simple_edgelist.feather"

print(f"Loading subset edgelist from: {edgelist_path}")

# Read the subset edgelist
edgelist = read_feather_gcs(edgelist_path, gcs_fs=gcs)

# Get unique neuron IDs from pre and post columns
subset_ids = set(edgelist['pre'].unique()) | set(edgelist['post'].unique())

print(f"Found {len(subset_ids):,} unique neurons in {SUBSET_NAME} subset")

Loading subset edgelist from: gs://sjcabs_2025_data/banc/front_leg/banc_746_front_leg_simple_edgelist.feather
📦 Loading from cache: sjcabs_2025_data_banc_front_leg_banc_746_front_leg_simple_edgelist.feather
✓ Loaded 348,856 rows (cached)
Found 3,532 unique neurons in front_leg subset


---

## Read Meta Data

In [6]:
# Load full metadata
meta_full = read_feather_gcs(meta_path, gcs_fs=gcs)

# Filter metadata to neurons in this subset
meta = meta_full[meta_full[DATASET_ID].isin(subset_ids)].copy()

print(f"\nLoaded {len(meta):,} neurons in {SUBSET_NAME} subset")
print(f"(From {len(meta_full):,} total proofread neurons)")
meta.head()

📦 Loading from cache: sjcabs_2025_data_banc_banc_746_meta.feather
✓ Loaded 168,759 rows (cached)

Loaded 3,532 neurons in front_leg subset
(From 168,759 total proofread neurons)


,banc_746_id,supervoxel_id,region,side,hemilineage,nerve,flow,super_class,cell_class,cell_sub_class,cell_type,neurotransmitter_predicted,neurotransmitter_score,cell_function,cell_function_detailed,body_part_sensory,body_part_effector,status
73,720575941433075470,76001336759598072,neck_connective,right,00A,None,intrinsic,ascending,ascending_neuron,None,AN00A002,gaba,0.9556,None,None,None,None,"MANC_MATCH_WRONG_SIDE,NEEDS_FAFB_MATCH,REVIEW_..."
79,720575941449981277,76001336759835146,neck_connective,left,01A,None,intrinsic,ascending,ascending_neuron,None,AN01A001,acetylcholine,0.8979,None,None,None,None,"REEVAL_MANC_MATCH,REVIEW_MATCH_AN_DN,FAFB_MATC..."
81,720575941499772066,76001336759524206,neck_connective,left,01A,None,intrinsic,ascending,ascending_neuron,None,AN01A006,acetylcholine,0.9554,None,None,None,None,"MANC_MATCH_WRONG_SIDE,NEEDS_FAFB_MATCH,REVIEW_..."
83,720575941495128446,76001336827286846,neck_connective,left,01A,None,intrinsic,ascending,ascending_neuron,None,AN01A014,acetylcholine,0.9425,None,None,None,None,"MANC_MATCH_WRONG_SIDE,NEEDS_FAFB_MATCH,REVIEW_..."
84,720575941541480573,76282811803998996,neck_connective,right,01A,None,intrinsic,ascending,ascending_neuron,None,AN01A014,acetylcholine,0.9558,None,None,None,None,"MANC_MATCH_WRONG_SIDE,INVESTIGATE,NO_FAFB_MATC..."


### Explore the Dataset

Let's examine the available neurons in this dataset:

### Examine Cell Types

What are the top cell types in this dataset?

In [7]:
# Count by cell type
cell_type_counts = meta['cell_type'].value_counts().head(15)

print("Top 15 cell types in dataset:")
print(cell_type_counts)

# Create bar plot
fig = go.Figure()

fig.add_trace(go.Bar(
    x=cell_type_counts.index,
    y=cell_type_counts.values,
    marker_color='steelblue',
    text=cell_type_counts.values,
    textposition='outside'
))

fig.update_layout(
    title=f"Top Cell Types: {DATASET}",
    xaxis_title="Cell Type",
    yaxis_title="Count",
    template="plotly_white",
    height=500,
    xaxis_tickangle=-45
)

fig.show()

Top 15 cell types in dataset:
cell_type
SNxxxx     109
SNta29      59
SNta44      58
SNta30      47
SNta43      43
SNch05      31
SNppxx      25
SNta21      23
SNta40      21
SNta34      19
SNch09      19
SNta42      18
SNch14f     17
SNta32      16
SNpp39      15
Name: count, dtype: int64


---

## Read Neuron Skeletons (.swc files)

Neuron skeletons are stored as SWC files in the GCS bucket. We'll use navis to read them.

### Load a Single Neuron

Let's start by loading one DNg12 neuron:

In [8]:
# Filter for DNg12 neurons from full metadata
meta_dng12 = meta[
    meta['cell_type'].str.contains('DNg12', case=False, na=False)
]

# Read ALL DNg12 neurons
if len(meta_dng12) > 0:
    # Get all DNg12 neuron IDs
    neuron_ids = meta_dng12[DATASET_ID].values
    print(f"Found {len(neuron_ids)} DNg12 neurons")
    
    # Create filenames
    filenames = [f"{nid}.swc" for nid in neuron_ids]
    
    # Read neurons
    if USE_GCS:
        gcs_skeleton_path = skeletons_path.replace("gs://", "")
        neurons = batch_read_swc_from_gcs(
            gcs,
            gcs_skeleton_path,
            filenames,
            show_progress=False
        )
    else:
        neurons = navis.NeuronList([
            navis.read_swc(f"{skeletons_path}/{fname}")
            for fname in filenames
        ])
    
    # Set neuron IDs as names for plotly hover labels
    for j, neuron_id in enumerate(neuron_ids):
        neurons[j].name = str(neuron_id)

    print(f"✓ Loaded {len(neurons)} DNg12 neurons")
else:
    print("No DNg12 neurons found")
    neurons = navis.NeuronList([])

Found 16 DNg12 neurons


✓ Loaded 16 DNg12 neurons


### 3D Visualization with Plotly

navis makes it easy to visualize neurons in 3D with plotly backend:

In [9]:
# Plot a single DNg12 neuron example
if len(neurons) > 0:
    # Select first neuron
    neuron = neurons[0]
    
    # Plot with navis (neurons are in nm, same as mesh coordinates)
    navis.plot3d(
        neuron,
        backend='plotly',
        color='darkblue',
        width=1200,
        height=800,
        title=f"DNg12 Neuron Example (1 of {len(neurons)})"
    )
else:
    print("No neurons to plot")

---

## Read Neuropil Meshes

Neuropil meshes are stored as OBJ files. We can load them and visualize neurons in their anatomical context.

In [10]:
# Step 1: Load large region meshes (brain + VNC) from obj/ directory
region_path = f"{DATA_PATH}/{dataset_base}/obj"
print(f"Large region meshes path: {region_path}")

if USE_GCS:
    gcs_region_path = region_path.replace("gs://", "")
    region_files = gcs.ls(gcs_region_path)
    region_files = [f"gs://{f}" for f in region_files if f.endswith('.obj')]
else:
    import glob
    region_files = glob.glob(f"{region_path}/*.obj")

print(f"Found {len(region_files)} region mesh files")

# Find VNC and brain meshes
vnc_files = [f for f in region_files if 'vnc' in f.lower()]
brain_files = [f for f in region_files if 'brain' in f.lower()]

# Step 2: Load specific neuropil meshes from obj/neuropils/
neuropil_path = f"{DATA_PATH}/{dataset_base}/obj/neuropils"
print(f"Neuropil meshes path: {neuropil_path}")

if USE_GCS:
    gcs_neuropil_path = neuropil_path.replace("gs://", "")
    neuropil_files = gcs.ls(gcs_neuropil_path)
    neuropil_files = [f"gs://{f}" for f in neuropil_files if f.endswith('.obj')]
else:
    neuropil_files = glob.glob(f"{neuropil_path}/*.obj")

# Find T1 leg neuropil mesh
leg_files = [f for f in neuropil_files if re.search(NEUROPIL_PATTERN, f, re.IGNORECASE)]

# Collect meshes to load
meshes_to_load = []
mesh_labels = []

if brain_files:
    meshes_to_load.append(brain_files[0])
    mesh_labels.append('brain')

if vnc_files:
    meshes_to_load.append(vnc_files[0])
    mesh_labels.append('vnc')

if leg_files:
    meshes_to_load.append(leg_files[0])
    mesh_labels.append('leg')

print(f"Total meshes to load: {len(meshes_to_load)}")

# Store for later use
obj_files = meshes_to_load  # Keep for compatibility with next cell


Large region meshes path: gs://sjcabs_2025_data/banc/obj
Found 3 region mesh files
Neuropil meshes path: gs://sjcabs_2025_data/banc/obj/neuropils


Total meshes to load: 3


### Load and Visualize Multiple Neurons with Neuropil Context

Let's load VNC and leg neuropil meshes and visualize multiple DNg12 neurons in their anatomical context, each in a different color:

In [11]:
# Load the meshes
loaded_meshes = []
for mesh_path in meshes_to_load:
    try:
        if USE_GCS:
            mesh = read_obj_from_gcs(gcs, mesh_path.replace('gs://', ''))
        else:
            mesh = trimesh.load(mesh_path)
        loaded_meshes.append(mesh)
        print(f"✓ Loaded: {mesh_path.split('/')[-1]}")
    except Exception as e:
        print(f"✗ Failed to load {mesh_path}: {e}")

print(f"\n✓ Loaded {len(loaded_meshes)} meshes")

# Convert trimesh meshes to navis.Volume objects for visualization
if len(meta_dng12) > 0 and len(loaded_meshes) > 0:
    print(f"\nVisualizing {len(neurons)} DNg12 neurons with {len(loaded_meshes)} meshes using navis...")

    # Convert loaded_meshes to navis.Volume objects with color set directly
    # CRITICAL: Volumes don't consume color entries from plot3d()
    # Set color directly on Volume objects using RGBA tuple
    mesh_volumes = []

    for i, (mesh, label) in enumerate(zip(loaded_meshes, mesh_labels)):
        # Set alpha based on label
        if label == 'leg':
            alpha = 0.3  # Leg more visible
        else:  # brain or vnc
            alpha = 0.1  # Very transparent

        # Create navis.Volume with color set directly
        # lightgrey RGB = (0.827, 0.827, 0.827)
        volume = navis.Volume(
            mesh.vertices,
            mesh.faces,
            name=label,
            color=(0.827, 0.827, 0.827, alpha)
        )
        mesh_volumes.append(volume)

    # Create NeuronList
    neuronlist = navis.NeuronList(neurons)

    # Plot with navis using correct pattern: [volumes..., neuronlist]
    # CRITICAL: Volumes don't consume color entries from plot3d()
    # Only pass colors for NEURONS (navis unpacks NeuronList)
    plot_objects = mesh_volumes + [neuronlist]
    neuron_colors = ['red'] * len(neuronlist)  # One color per neuron
    neuron_alphas = [1.0] * len(neuronlist)   # One alpha per neuron

    fig = navis.plot3d(
        plot_objects,
        color=neuron_colors,  # ONLY neuron colors!
        alpha=neuron_alphas,   # ONLY neuron alphas!
        backend='plotly',
        width=1400,
        height=900,
        title=f'{len(neurons)} DNg12 Neurons with Neuropil Context'
    )

    if fig is not None:
        fig.update_layout(
            scene=dict(
                xaxis_title='X (nm)',
                yaxis_title='Y (nm)',
                zaxis_title='Z (nm)',
                aspectmode='data'
            )
        )
        fig.show()
    else:
        print("Note: Plot was displayed inline or could not be generated as a figure object")
else:
    print("⚠ No neurons or meshes to visualize")


---

## Co-plotting Neurons Across Datasets

One powerful feature of our data organization is that we provide neuron skeletons in both native space AND in BANC space. This enables easy co-visualization of neurons from different datasets.

Let's load EPG neurons from the **maleCNS** dataset (which are already transformed to BANC space) and co-plot them with our BANC EPG neurons:

In [12]:
# Switch to maleCNS dataset
dataset_male = "malecns_09"
dataset_male_id = "malecns_09_id"

# Read maleCNS meta data
meta_male_path = construct_path(DATA_PATH, dataset_male, "meta")
meta_male_full = read_feather_gcs(meta_male_path, gcs_fs=gcs)

# Filter for DNg12 neurons in maleCNS
meta_male_dng12 = meta_male_full[
    meta_male_full['cell_type'].str.contains('DNg12', case=False, na=False)
]

print(f"Found {len(meta_male_dng12)} DNg12 neurons in maleCNS")

if len(meta_male_dng12) > 0:
    # Construct path to maleCNS skeletons in BANC space
    skeletons_male_banc_path = construct_path(
        DATA_PATH,
        dataset_male,
        "skeletons",
        space_suffix="banc_space"
    )

    # Read maleCNS neurons (already in BANC space)
    male_ids = meta_male_dng12[dataset_male_id].values
    print(f"Reading {len(male_ids)} maleCNS neurons from BANC space...")

    male_filenames = [f"{nid}.swc" for nid in male_ids]

    if USE_GCS:
        gcs_male_path = skeletons_male_banc_path.replace("gs://", "")
        neurons_male_banc = batch_read_swc_from_gcs(
            gcs,
            gcs_male_path,
            male_filenames,
            show_progress=False
        )
    else:
        neurons_male_banc = navis.NeuronList([
            navis.read_swc(f"{skeletons_male_banc_path}/{fname}")
            for fname in male_filenames
        ])

    print(f"✓ Loaded {len(neurons_male_banc)} maleCNS neurons")

    # Co-plot BANC and maleCNS neurons using navis.plot3d()
    if len(meta_dng12) > 0 and len(neurons) > 0:
        # Prepare volumes
        mesh_volumes = []
        if 'loaded_meshes' in locals() and len(loaded_meshes) > 0:
            for i, mesh in enumerate(loaded_meshes):
                # Set color directly on Volume (lightgrey with low alpha)
                mesh_volume = navis.Volume(
                    mesh.vertices,
                    mesh.faces,
                    name=f'Neuropil {i+1}',
                    color=(0.827, 0.827, 0.827, 0.1)  # lightgrey RGBA
                )
                mesh_volumes.append(mesh_volume)

        # Create NeuronLists
        banc_neuronlist = navis.NeuronList(neurons[:3])
        male_neuronlist = navis.NeuronList(neurons_male_banc[:3])

        # Plot with navis using list pattern
        # CRITICAL: Volumes don't consume color entries from plot3d()
        # Only pass colors for neurons (navis unpacks NeuronLists)
        plot_objects = mesh_volumes + [banc_neuronlist, male_neuronlist]
        banc_colors = ['navy'] * len(banc_neuronlist)
        male_colors = ['red'] * len(male_neuronlist)
        neuron_colors = banc_colors + male_colors  # ONLY neuron colors

        # Alpha only for neurons (not volumes)
        banc_alphas = [1.0] * len(banc_neuronlist)
        male_alphas = [1.0] * len(male_neuronlist)
        neuron_alphas = banc_alphas + male_alphas

        fig = navis.plot3d(
            plot_objects,
            color=neuron_colors,  # ONLY neuron colors!
            alpha=neuron_alphas,   # ONLY neuron alphas!
            backend='plotly',
            width=1200,
            height=800,
            title="DNg12 Neurons: BANC (navy) vs maleCNS (red) in BANC Space"
        )

        if fig is not None:
            fig.update_layout(
                scene=dict(
                    xaxis_title='X (nm)',
                    yaxis_title='Y (nm)',
                    zaxis_title='Z (nm)',
                    aspectmode='data'
                )
            )
            fig.show()
else:
    print("No DNg12 neurons found in maleCNS")


---

## Load Multiple Neurons for NBLAST

Let's load neurons from the ventral nerve cord for morphological comparison. We'll filter for intrinsic VNC neurons restricted to a single leg neuromere, and take one example per cell type for computational efficiency:

In [13]:
# Filter for intrinsic VNC neurons in single leg neuromere
meta_vnc = meta[
    (meta['super_class'] == 'ventral_nerve_cord_intrinsic') &
    (meta['cell_class'] == 'single_leg_neuromere')
].copy()

print(f"Found {len(meta_vnc)} VNC intrinsic neurons in single leg neuromere")

# Sample 100 neurons for morphometric analysis
# Note: Sampling is for tutorial performance, not usually done in real analysis
n_sample = min(100, len(meta_vnc))
meta_vnc_sampled = meta_vnc.sample(n=n_sample, random_state=42)

print(f"Sampled {len(meta_vnc_sampled)} neurons for analysis")
print(f"\nTop cell types:")
print(meta_vnc_sampled['cell_type'].value_counts().head(10))

# Get neuron IDs
sampled_ids = meta_vnc_sampled[DATASET_ID].values

if len(sampled_ids) > 0:
    print(f"\nLoading {len(sampled_ids)} neurons for NBLAST analysis...")
    
    # Prepare filenames
    swc_filenames = [f"{nid}.swc" for nid in sampled_ids]
    
    # Batch read from GCS
    if USE_GCS:
        gcs_skeleton_path = skeletons_path.replace("gs://", "")
        neurons = batch_read_swc_from_gcs(gcs, gcs_skeleton_path, swc_filenames, show_progress=True)

        # Set neuron IDs as names for plotly hover labels
        for j, neuron_id in enumerate(sampled_ids):
            neurons[j].name = str(neuron_id)
    else:
        # Local reading
        neurons = []
        for fname in tqdm(swc_filenames):
            try:
                n = navis.read_swc(f"{skeletons_path}/{fname}")
                neurons.append(n)
            except Exception as e:
                print(f"Error reading {fname}: {e}")
        neurons = navis.NeuronList(neurons)
    
    print(f"\n✓ Loaded {len(neurons)} neurons successfully")
else:
    print("No neurons found matching the filter criteria")
    neurons = navis.NeuronList([])

Found 484 VNC intrinsic neurons in single leg neuromere
Sampled 100 neurons for analysis

Top cell types:
cell_type
IN23B028    3
IN21A087    3
IN01A079    2
IN03A075    2
IN03A062    2
IN03A073    2
IN14B005    2
IN12B059    2
IN12B038    2
IN13B058    1
Name: count, dtype: int64

Loading 100 neurons for NBLAST analysis...


Reading neurons:   0%|          | 0/100 [00:00<?, ?it/s]


✓ Loaded 100 neurons successfully


### Morphometric Analysis

Calculate morphological properties for all neurons:

In [14]:
# Convert to microns
neurons_um = neurons / 1000

print(f"\nComputing Strahler orders for {len(neurons_um)} neurons...")

# Calculate Strahler index for all neurons
# This adds a 'strahler_index' column to each neuron's nodes DataFrame
for n in neurons_um:
    navis.morpho.strahler_index(n)

print("✓ Strahler indices computed!")

# Collect Strahler statistics for each neuron
strahler_stats = []
for n in neurons_um:
    if 'strahler_index' in n.nodes.columns:
        max_order = n.nodes['strahler_index'].max()
        strahler_stats.append({
            'neuron_id': n.id,
            'max_strahler_order': max_order,
            'n_nodes': n.n_nodes,
            'cable_length_um': n.cable_length
        })

# Create DataFrame
strahler_df = pd.DataFrame(strahler_stats)

# Add cell type information
cell_type_map = dict(zip(meta_vnc_sampled[DATASET_ID].values, 
                         meta_vnc_sampled['cell_type'].values))
strahler_df['cell_type'] = strahler_df['neuron_id'].map(cell_type_map)

print(f"\nStrahler Order Summary:")
print(f"  Neurons analyzed: {len(strahler_df)}")
print(f"  Max Strahler order range: {strahler_df['max_strahler_order'].min():.0f} - {strahler_df['max_strahler_order'].max():.0f}")
print(f"  Mean max Strahler order: {strahler_df['max_strahler_order'].mean():.1f} ± {strahler_df['max_strahler_order'].std():.1f}")

strahler_df.head(10)


Computing morphological properties for 100 neurons...
⏳ This may take a few minutes for complex neurons...

1/4: Computing cable lengths...


Cable length:   0%|          | 0/100 [00:00<?, ?it/s]

2/4: Counting nodes...


Nodes:   0%|          | 0/100 [00:00<?, ?it/s]

3/4: Counting branches...


Branches:   0%|          | 0/100 [00:00<?, ?it/s]

4/4: Counting leaf nodes...


Leafs:   0%|          | 0/100 [00:00<?, ?it/s]


✓ Morphological properties computed!

Morphological Summary:
       cable_length_um      n_nodes  n_branches     n_leafs
count       100.000000   100.000000  100.000000  100.000000
mean       2091.616455   542.960000  118.020000  309.900000
std        1310.047852   349.427755   72.651094  226.047885
min         419.356750    92.000000   15.000000   29.000000
25%        1120.100464   296.000000   65.750000  149.000000
50%        1535.785950   399.000000   88.500000  208.000000
75%        2916.190369   770.750000  163.250000  428.250000
max        5939.402344  1530.000000  322.000000  996.000000


,neuron_id,cell_type,cable_length_um,n_nodes,n_branches,n_leafs
0,720575941464268118,IN20A.22A055,680.186768,163,35,83
1,720575941518483054,IN23B054,1476.030762,345,71,185
2,720575941438885723,IN12B038,1072.992432,262,58,109
3,720575941661454904,IN21A023,1756.054810,509,116,274
4,720575941555229953,IN14A024,1302.663574,349,74,184
5,720575941654986201,IN09B008,3854.221680,1016,235,599
6,720575941588628620,IN19A088,1087.564941,280,65,147
7,720575941689865880,IN19B012,4809.693359,1307,241,882
8,720575941496088446,IN26X002,3600.500000,911,189,528
9,720575941501229143,IN13B006,3053.548828,790,158,497


### Strahler Order Analysis

**Strahler order** is a numerical measure of branching complexity that classifies branches hierarchically:

<p align="center">
  <img src="../inst/images/strahler_order.png" alt="Strahler Order Diagram" width="70%">
</p>

**Classification rules:**
- **Order 1**: Terminal branches (no children)
- **Order n+1**: Parent of two order-n branches
- **Higher order**: Parent takes the maximum order of its children when orders differ

This metric reveals:
- **Overall complexity**: Maximum Strahler order indicates branching hierarchy depth
- **Cable distribution**: How much cable is allocated to different branching levels
- **Arbor architecture**: Balance between terminal branches (order 1) and main trunks (high orders)

Let's visualize the Strahler order distribution across our VNC neurons:

In [15]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set seaborn style
sns.set_style("whitegrid")
sns.set_palette("Set2")

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Distribution of maximum Strahler orders
ax1 = axes[0, 0]
sns.histplot(
    data=strahler_df,
    x='max_strahler_order',
    bins=range(int(strahler_df['max_strahler_order'].min()), 
               int(strahler_df['max_strahler_order'].max()) + 2),
    ax=ax1,
    kde=False
)
ax1.set_xlabel('Maximum Strahler Order', fontsize=12)
ax1.set_ylabel('Number of Neurons', fontsize=12)
ax1.set_title('Distribution of Maximum Strahler Orders', fontsize=13, fontweight='bold')

# 2. Strahler order vs cable length
ax2 = axes[0, 1]
sns.scatterplot(
    data=strahler_df,
    x='max_strahler_order',
    y='cable_length_um',
    ax=ax2,
    alpha=0.6
)
ax2.set_xlabel('Maximum Strahler Order', fontsize=12)
ax2.set_ylabel('Cable Length (µm)', fontsize=12)
ax2.set_title('Strahler Order vs Cable Length', fontsize=13, fontweight='bold')

# 3. Strahler order vs number of nodes
ax3 = axes[1, 0]
sns.scatterplot(
    data=strahler_df,
    x='max_strahler_order',
    y='n_nodes',
    ax=ax3,
    alpha=0.6,
    color='coral'
)
ax3.set_xlabel('Maximum Strahler Order', fontsize=12)
ax3.set_ylabel('Number of Nodes', fontsize=12)
ax3.set_title('Strahler Order vs Node Count', fontsize=13, fontweight='bold')

# 4. Box plot of Strahler orders
ax4 = axes[1, 1]
sns.boxplot(
    data=strahler_df,
    x='max_strahler_order',
    ax=ax4
)
ax4.set_xlabel('Maximum Strahler Order', fontsize=12)
ax4.set_ylabel('', fontsize=12)
ax4.set_title('Strahler Order Variability', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n" + "="*60)
print("Strahler Order Analysis Summary")
print("="*60)
print(f"\nNeurons by maximum Strahler order:")
order_counts = strahler_df['max_strahler_order'].value_counts().sort_index()
for order, count in order_counts.items():
    pct = (count / len(strahler_df)) * 100
    print(f"  Order {int(order)}: {count} neurons ({pct:.1f}%)")

print(f"\nMorphological complexity:")
print(f"  Mean max order: {strahler_df['max_strahler_order'].mean():.2f}")
print(f"  Std dev: {strahler_df['max_strahler_order'].std():.2f}")
print(f"  Range: {int(strahler_df['max_strahler_order'].min())} - {int(strahler_df['max_strahler_order'].max())}")

---

## NBLAST Morphological Similarity

**NBLAST** (Neuron BLAST) is an algorithm for comparing neuron morphology based on local geometry. It works by:

1. Converting neurons to point clouds with tangent vectors (dotprops)
2. For each point in query neuron, finding nearest point in target
3. Computing similarity based on distance and angle between tangent vectors

navis includes a fast NBLAST implementation (uses compiled Rust code under the hood).

**Note:** For this tutorial, we'll demonstrate the code but not execute it since it requires additional computation time. Remove `eval=False` to run it yourself.

In [16]:
# Convert neurons to dotprops (point clouds with tangent vectors)
# This removes connectivity but preserves spatial structure
neurons_dp = navis.make_dotprops(neurons_um, k=5)

print(f"  {len(neurons_dp)} neurons ready for NBLAST")

# Run NBLAST all-by-all comparison
# When query and target are the same, performs all-by-all
# Use all available cores (minus 1 to keep system responsive)
import multiprocessing
n_cores = max(1, multiprocessing.cpu_count() - 1)

print(f"\nRunning NBLAST using {n_cores} cores (this may take a moment)...")
nblast_scores = navis.nblast(neurons_dp, neurons_dp, n_cores=n_cores)

print(f"Score matrix shape: {nblast_scores.shape}")

# Display score matrix
nblast_scores.head()

  100 neurons ready for NBLAST

Running NBLAST using 9 cores (this may take a moment)...


Preparing:   0%|          | 0/1 [00:00<?, ?it/s]

NBlasting:   0%|          | 0/100 [00:00<?, ?it/s]

Score matrix shape: (100, 100)


target,2c41f166-251f-4ec2-ab54-ab0b7203493a,0b85c33c-11b8-47f5-b34c-9a00b3306c70,225e18ab-6b9d-4466-949a-9ef291d96a07,33ac6300-e546-4fa0-952c-f361c95b37b1,b7f30f99-413b-45e7-a4f6-7376d3e5fb05,95b0893d-1fa8-4550-b503-39175138e536,2b665dcf-c413-46da-8c9d-fadce7ed13e8,5b38351e-0ca3-4ef9-b144-4f6e50f8f0c7,c8c55025-681c-4afe-9755-6a6b1d9bc21f,4ee1c771-ccd3-4efd-9a65-2a202142869b,84c7956e-bff7-447a-996c-84b3407e8e3c,10c90a8a-7af0-426f-9c82-c251b56c811a,3b0120b5-deb1-4fbe-a4f3-ffc5ed63c741,e5231dfb-7fb1-4a3c-b09c-601e220a3b17,134e23c2-e08c-40b5-8de3-64b53f522b24,740c7a9e-1bb1-48e0-b56a-22e92d99a4dc,233263cd-f072-4720-ba54-555b919a8f46,fde3a17d-64a7-4de5-94e1-c856c486793e,41fb5147-165c-4c6f-8b56-68085ff79d58,a3f3e9ae-f132-40ad-88f1-8770da179835,f0cd7688-cc8d-4577-aaec-7469e7af3528,d075223f-1eb7-49d0-b4d5-19fc6cae2de4,c2324364-83fe-4624-b208-42ed73ebc6dd,d527fd73-eb30-4b13-affb-afa2c0bff907,8d5c28f0-1184-4b0d-ad25-5b949b766ac9,3c4b72ac-f028-48f7-acae-589885a7c670,cdcbdbe4-c384-469f-8d79-c0a87bf93aa8,5f3ec3cc-474f-4dac-9fcd-869305a02a9b,45932e42-e256-4551-a2e3-83ebbd4c848d,614745bf-6de0-4eaf-a39a-e09866b153a5,74749bc0-69e7-4dc0-a69b-cee9cae5b68f,ea9cfa6d-906f-429b-8ecc-37db2d6770d6,a14bc033-1d21-4f39-912b-cba14834a9ae,59da4789-4bdc-44f3-a4fc-afc2484068fd,2b702bb8-3170-4f02-ab6a-09214be5dd14,87480af1-45cc-4178-9a3a-a8a504f55a04,6eecdcfc-68e0-41ce-8e45-c6a237c12e13,32a38d29-3246-4469-b392-806df81c44a7,20c653cd-3a27-4d99-800b-1737493b265f,83100037-5e30-488b-9da9-d47910da4db4,e77bd8ea-bea7-4062-890a-324c1788b6f6,11d676d3-3f6f-4223-ad39-4c8d7f99cdfb,8e589366-2d8e-4cb1-8ae2-3c371ac77925,5228c8f4-d118-44ea-aa92-5f3fc23abcfc,278b0b18-2fc0-42e8-8c93-0ad799fbda13,9c1e2589-1559-4178-b4d9-ced433235c41,e3b42d16-ffec-4c5b-8380-9788b526dba4,7acd8257-5094-4825-b8f9-13370667c230,e65a894f-aa41-4986-8b89-a25753d21147,4a34d8d2-425a-49c1-8b69-e190107e5a86,5a427f5c-69f0-46f7-b703-3549f69efdeb,3354f4d9-ab81-446a-8f1d-25cca3597e7f,6ca63252-0939-401d-acd5-a269303ceb2a,8a5fba00-21bb-4ebe-a5cf-9deef41b8c6e,2d08487c-1e3a-4b6b-9ffe-7e4edc44e4b5,72267e7d-41ee-4769-827d-260266e98b67,f3b660a2-ddff-4836-aace-73aa9f4e6a8a,b45b91a0-c67f-4dfa-97e1-214c35841839,e7e71a99-4d00-4ab6-8b36-47d95424acab,54030fd2-f6a7-40b9-9540-8df0487c2e7c,75e527d7-7e82-4dae-8b98-dbbb4f6f404c,2aa6d4d9-4b40-4cf5-9d24-ad368fc2e35b,e992010b-2f8f-4969-9a7c-d22e5ff6e928,9b6a8ae7-abe4-41d7-b672-08a36d2a09f7,615aefb8-b0ba-444d-a1ff-610b83eec02f,aea5f6e7-c881-482d-8621-46b1f9ed237f,daa1c152-d540-459f-875b-536c2bc8e001,94ecebcc-e394-4343-8944-35ae1e86bc1e,480b41a7-3cc9-4939-8ebe-07d38612e7f3,731607aa-7bbf-4fb3-a771-8b0ece758950,6c498aaa-669d-4a4c-b895-0930dfb96a17,c3e39c28-fd6b-42ef-bf32-0e9021412a37,ba84ea91-e618-43ce-83cc-73ba6bd23399,e954f218-9560-449f-9c03-d3225b14b6dc,474d7ab3-d8f8-48b1-9cf6-b30222541afa,32659f13-0392-468d-892a-f851522be3f6,fd4ea3be-6c1a-4bfc-8c25-4059639cb2a3,16b6eca4-eee9-42b9-9f32-efdfa7842b64,7eeed472-79d9-4d52-b1e5-e03745c051a2,14805af8-f7d8-4ba5-8a45-25fbf33d63d8,0876da1c-faf7-44d3-82b3-e1c0407148cd,874346e2-8b00-44f0-b985-8f73fe95cfbd,3b5e97d4-090e-4242-b878-7fe48a37063e,d076bf2c-c4b3-4dbf-a751-4319003c214e,f49ffb33-5e26-46ca-8b9f-ddd4b7d1d439,576560fd-9c93-40ba-afbc-c7567c784c75,1d2eaec2-c74e-4199-adac-4ff58bc6c5bc,1ad77b9c-b0cf-4f76-9445-1c6f0af79f37,e16bd987-da02-48a2-8809-e934784d2182,06255905-b7f8-45bf-b734-83d7fea3910a,ba0cd410-159c-4258-bd13-cf051ec2de89,f3b0b213-ca8f-46b0-a12f-01db2deee2be,10b64fcb-9110-4759-aa76-896ac5153b33,c7b93c15-b81c-4fbe-9e1e-b7b21ae32ef7,0e907964-be7b-4933-9b77-5f69b772b978,a44abf49-8bc1-46e7-b19c-44a863faa91a,99412c9d-3b06-411e-a720-2f91bd46d33e,f3f6de46-fcc8-4989-b773-fbf54271aa45,6ddfa261-6c39-4eb6-8d00-378e71e70545,3ab0874e-e2da-4217-b405-f9f6691d581e
query,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2c41f166-251f-4ec2-ab54-ab0b7203493a,1.000000,-0.346716,-0.262772,0.305059,-0.012930,-0.649961,-0.239373,0.141357,-0.066899,0.054600,-0.061565,-0.319375,0.471608,-0.079839,0.165943,0.4262

### Hierarchical Clustering

Use NBLAST scores to cluster neurons by morphological similarity:

In [17]:
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree
from scipy.spatial.distance import squareform
import plotly.figure_factory as ff

# Convert NBLAST scores to distance matrix
# NBLAST scores are similarity scores (higher = more similar)
# We need to normalize by self-scores and convert to distances
self_scores = np.diag(nblast_scores.values)
normalized_scores = nblast_scores.values / self_scores.max()
distance_matrix = 1 - normalized_scores

# Convert to condensed distance matrix for linkage
# Only use upper triangle (distance matrix is symmetric)
condensed_dist = squareform(distance_matrix, checks=False)

# Perform hierarchical clustering
linkage_matrix = linkage(condensed_dist, method='ward')

# Choose number of clusters
k = 8
print(f"Cutting dendrogram into {k} clusters...")

# Cut tree to get cluster assignments
clusters = cut_tree(linkage_matrix, n_clusters=k).flatten() + 1  # +1 to start from 1

# Get cut height for k clusters
# Height at which we cut is at the (n-k)th merge
cut_height = linkage_matrix[len(linkage_matrix) - k, 2]

print(f"Cut height for k={k}: {cut_height:.4f}")
print(f"\nCluster sizes:")
cluster_counts = pd.Series(clusters).value_counts().sort_index()
print(cluster_counts)

# Create dendrogram with plotly
# First create using scipy to get dendrogram coordinates
dend = dendrogram(linkage_matrix, no_plot=True)

# Get leaf order and corresponding clusters
leaf_order = dend['leaves']
leaf_clusters = clusters[leaf_order]

# Get cell types for leaves (map back through leaf_order to original neurons)
leaf_cell_types = strahler_df['cell_type'].iloc[leaf_order].values

# Create color map for clusters
import plotly.express as px
colors_list = px.colors.qualitative.Light24 + px.colors.qualitative.Dark24
cluster_colors = {i: colors_list[i % len(colors_list)] for i in range(1, k+1)}
leaf_colors = [cluster_colors[c] for c in leaf_clusters]

# Create figure
fig = go.Figure()

# Add dendrogram segments
for i, (x, y) in enumerate(zip(dend['icoord'], dend['dcoord'])):
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='lines',
        line=dict(color='grey', width=1),
        hoverinfo='skip',
        showlegend=False
    ))

# Add colored points at leaves with cell_type hover text
leaf_x = [(dend['icoord'][i][1] + dend['icoord'][i][2]) / 2 
          for i in range(len(dend['icoord'])) 
          if dend['dcoord'][i][0] == 0]
leaf_y = [0] * len(leaf_x)

# Group leaves by cluster for legend, including cell_type in hover text
for cluster_id in sorted(cluster_counts.index):
    cluster_mask = leaf_clusters == cluster_id
    cluster_x = [x for i, x in enumerate(leaf_x) if cluster_mask[i]]
    cluster_y = [0] * len(cluster_x)
    
    # Get cell types for this cluster's leaves
    cluster_cell_types = [leaf_cell_types[i] for i, mask in enumerate(cluster_mask) if mask]
    
    fig.add_trace(go.Scatter(
        x=cluster_x,
        y=cluster_y,
        mode='markers',
        marker=dict(size=8, color=cluster_colors[cluster_id]),
        name=f'Cluster {cluster_id}',
        hovertext=[f'Cell Type: {ct}<br>Cluster: {cluster_id}' for ct in cluster_cell_types],
        hoverinfo='text'
    ))

# Add horizontal cut line
fig.add_hline(
    y=cut_height,
    line=dict(color='red', width=2, dash='dash'),
    annotation_text=f'Cut height (k={k})',
    annotation_position='right'
)

# Update layout
fig.update_layout(
    title=dict(
        text=f'Hierarchical Clustering of Neuron Morphology (NBLAST)<br><sub>{DATASET} - {SUBSET_NAME} (k={k} clusters)</sub>',
        x=0.5,
        xanchor='center'
    ),
    xaxis=dict(
        title='Neurons',
        showticklabels=False,
        showgrid=False
    ),
    yaxis=dict(
        title='Height (Ward Distance)',
        showgrid=True,
        gridcolor='lightgrey'
    ),
    plot_bgcolor='white',
    width=1200,
    height=600,
    hovermode='closest',
    legend=dict(
        orientation='v',
        yanchor='top',
        y=1,
        xanchor='left',
        x=1.01,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='grey',
        borderwidth=1
    )
)

# Save static version
fig.write_image(f"{IMG_DIR}/{DATASET}_{SUBSET_NAME}_nblast_dendrogram.png", 
                width=1400, height=700, scale=2)
print(f"\n✓ Dendrogram saved to {IMG_DIR}/{DATASET}_{SUBSET_NAME}_nblast_dendrogram.png")

# Display interactive version
fig.show()

Cutting dendrogram into 8 clusters...
Cut height for k=8: 2.0031

Cluster sizes:
1    36
2     9
3     6
4    20
5    14
6     5
7     7
8     3
Name: count, dtype: int64



✓ Dendrogram saved to images/tutorial_02/banc_746_front_leg_nblast_dendrogram.png


### Interactive Cluster Visualization

Now let's create a single interactive 3D plot showing all clusters together. Each cluster is a separate trace that can be toggled on/off by clicking its legend entry:

In [18]:
# Create unified interactive plot with all clusters
# Convert leg mesh to navis.Volume for proper visualization
leg_volume = None
if 'loaded_meshes' in locals() and len(loaded_meshes) > 2:
    leg_mesh = loaded_meshes[2]  # Third mesh is leg neuropil
    leg_volume = navis.Volume(leg_mesh.vertices, leg_mesh.faces, name='Leg Neuropil', color=(0.827, 0.827, 0.827, 0.3))

# Collect cluster neurons (in nm, not µm!)
all_cluster_neurons = []
all_cluster_colors = []

for cluster_id in sorted(cluster_counts.index):
    # Get neurons in this cluster (using neurons in nm, not neurons_um!)
    cluster_mask = clusters == cluster_id
    cluster_neurons_list = [neurons[i] for i in range(len(neurons)) if cluster_mask[i]]

    if len(cluster_neurons_list) > 0:
        all_cluster_neurons.extend(cluster_neurons_list)
        all_cluster_colors.extend([cluster_colors[cluster_id]] * len(cluster_neurons_list))

# Create NeuronList for all cluster neurons
all_cluster_neuronlist = navis.NeuronList(all_cluster_neurons)

if len(all_cluster_neuronlist) > 0:
    print(f"\nPlotting {len(all_cluster_neuronlist)} neurons in {k} clusters with neuropil context...")

    # Plot with navis using list pattern
    if leg_volume is not None:
        # CRITICAL: Volumes don't consume colors from plot3d()
        # Only pass neuron colors (navis unpacks NeuronList)
        neuron_alphas = [1.0] * len(all_cluster_neuronlist)

        fig = navis.plot3d(
            [leg_volume, all_cluster_neuronlist],
            color=all_cluster_colors,  # ONLY neuron colors!
            alpha=neuron_alphas,
            backend='plotly',
            width=1400,
            height=900,
            title=f'NBLAST Clusters: All {k} Clusters (Neurons in nm)'
        )
    else:
        fig = navis.plot3d(
            all_cluster_neuronlist,
            color=all_cluster_colors,
            backend='plotly',
            width=1400,
            height=900,
            title=f'NBLAST Clusters: All {k} Clusters (Neurons in nm)'
        )

    if fig is not None:
        # Update layout for nm coordinates
        fig.update_layout(
            scene=dict(
                xaxis_title='X (nm)',
                yaxis_title='Y (nm)',
                zaxis_title='Z (nm)',
                aspectmode='data'
            ),
            showlegend=False
        )
        fig.show()
else:
    print("No cluster neurons to plot")


### Visualize Individual Clusters

Let's visualize each cluster separately to examine morphological patterns within clusters.

In [19]:
# Visualize Cluster 1
cluster_1_neurons = [neurons[i] for i in range(len(neurons)) if clusters[i] == 1]

if len(cluster_1_neurons) > 0:
    print(f'Cluster 1: {len(cluster_1_neurons)} neurons')

    # Prepare volume if available
    if 'loaded_meshes' in locals() and len(loaded_meshes) > 2:
        leg_mesh = loaded_meshes[2]
        leg_volume = navis.Volume(leg_mesh.vertices, leg_mesh.faces, name='Leg Neuropil', color=(0.827, 0.827, 0.827, 0.3))

        # Create NeuronList and plot with volume
        cluster_1_neuronlist = navis.NeuronList(cluster_1_neurons)

        # CRITICAL: Volumes don't consume colors from plot3d()
        # Only pass neuron colors (navis unpacks NeuronList)
        neuron_colors = [cluster_colors[1]] * len(cluster_1_neuronlist)
        neuron_alphas = [1.0] * len(cluster_1_neuronlist)

        fig = navis.plot3d(
            [leg_volume, cluster_1_neuronlist],
            color=neuron_colors,  # ONLY neuron colors!
            alpha=neuron_alphas,
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 1 Morphology ({len(cluster_1_neurons)} neurons)'
        )
    else:
        # Plot neurons only if no volume available
        cluster_1_neuronlist = navis.NeuronList(cluster_1_neurons)

        fig = navis.plot3d(
            cluster_1_neuronlist,
            color=cluster_colors[1],
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 1 Morphology ({len(cluster_1_neurons)} neurons)'
        )

    if fig is not None:
        fig.update_layout(
            scene=dict(
                xaxis_title='X (nm)',
                yaxis_title='Y (nm)',
                zaxis_title='Z (nm)',
                aspectmode='data'
            )
        )
        fig.show()
else:
    print(f'No neurons in cluster 1')


In [20]:
# Visualize Cluster 2
cluster_2_neurons = [neurons[i] for i in range(len(neurons)) if clusters[i] == 2]

if len(cluster_2_neurons) > 0:
    print(f'Cluster 2: {len(cluster_2_neurons)} neurons')

    # Prepare volume if available
    if 'loaded_meshes' in locals() and len(loaded_meshes) > 2:
        leg_mesh = loaded_meshes[2]
        leg_volume = navis.Volume(leg_mesh.vertices, leg_mesh.faces, name='Leg Neuropil', color=(0.827, 0.827, 0.827, 0.3))

        # Create NeuronList and plot with volume
        cluster_2_neuronlist = navis.NeuronList(cluster_2_neurons)

        # CRITICAL: Volumes don't consume colors from plot3d()
        # Only pass neuron colors (navis unpacks NeuronList)
        neuron_colors = [cluster_colors[2]] * len(cluster_2_neuronlist)
        neuron_alphas = [1.0] * len(cluster_2_neuronlist)

        fig = navis.plot3d(
            [leg_volume, cluster_2_neuronlist],
            color=neuron_colors,  # ONLY neuron colors!
            alpha=neuron_alphas,
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 2 Morphology ({len(cluster_2_neurons)} neurons)'
        )
    else:
        # Plot neurons only if no volume available
        cluster_2_neuronlist = navis.NeuronList(cluster_2_neurons)

        fig = navis.plot3d(
            cluster_2_neuronlist,
            color=cluster_colors[2],
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 2 Morphology ({len(cluster_2_neurons)} neurons)'
        )

    if fig is not None:
        fig.update_layout(
            scene=dict(
                xaxis_title='X (nm)',
                yaxis_title='Y (nm)',
                zaxis_title='Z (nm)',
                aspectmode='data'
            )
        )
        fig.show()
else:
    print(f'No neurons in cluster 2')


In [21]:
# Visualize Cluster 3
cluster_3_neurons = [neurons[i] for i in range(len(neurons)) if clusters[i] == 3]

if len(cluster_3_neurons) > 0:
    print(f'Cluster 3: {len(cluster_3_neurons)} neurons')

    # Prepare volume if available
    if 'loaded_meshes' in locals() and len(loaded_meshes) > 2:
        leg_mesh = loaded_meshes[2]
        leg_volume = navis.Volume(leg_mesh.vertices, leg_mesh.faces, name='Leg Neuropil', color=(0.827, 0.827, 0.827, 0.3))

        # Create NeuronList and plot with volume
        cluster_3_neuronlist = navis.NeuronList(cluster_3_neurons)

        # CRITICAL: Volumes don't consume colors from plot3d()
        # Only pass neuron colors (navis unpacks NeuronList)
        neuron_colors = [cluster_colors[3]] * len(cluster_3_neuronlist)
        neuron_alphas = [1.0] * len(cluster_3_neuronlist)

        fig = navis.plot3d(
            [leg_volume, cluster_3_neuronlist],
            color=neuron_colors,  # ONLY neuron colors!
            alpha=neuron_alphas,
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 3 Morphology ({len(cluster_3_neurons)} neurons)'
        )
    else:
        # Plot neurons only if no volume available
        cluster_3_neuronlist = navis.NeuronList(cluster_3_neurons)

        fig = navis.plot3d(
            cluster_3_neuronlist,
            color=cluster_colors[3],
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 3 Morphology ({len(cluster_3_neurons)} neurons)'
        )

    if fig is not None:
        fig.update_layout(
            scene=dict(
                xaxis_title='X (nm)',
                yaxis_title='Y (nm)',
                zaxis_title='Z (nm)',
                aspectmode='data'
            )
        )
        fig.show()
else:
    print(f'No neurons in cluster 3')


In [22]:
# Visualize Cluster 4
cluster_4_neurons = [neurons[i] for i in range(len(neurons)) if clusters[i] == 4]

if len(cluster_4_neurons) > 0:
    print(f'Cluster 4: {len(cluster_4_neurons)} neurons')

    # Prepare volume if available
    if 'loaded_meshes' in locals() and len(loaded_meshes) > 2:
        leg_mesh = loaded_meshes[2]
        leg_volume = navis.Volume(leg_mesh.vertices, leg_mesh.faces, name='Leg Neuropil', color=(0.827, 0.827, 0.827, 0.3))

        # Create NeuronList and plot with volume
        cluster_4_neuronlist = navis.NeuronList(cluster_4_neurons)

        # CRITICAL: Volumes don't consume colors from plot3d()
        # Only pass neuron colors (navis unpacks NeuronList)
        neuron_colors = [cluster_colors[4]] * len(cluster_4_neuronlist)
        neuron_alphas = [1.0] * len(cluster_4_neuronlist)

        fig = navis.plot3d(
            [leg_volume, cluster_4_neuronlist],
            color=neuron_colors,  # ONLY neuron colors!
            alpha=neuron_alphas,
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 4 Morphology ({len(cluster_4_neurons)} neurons)'
        )
    else:
        # Plot neurons only if no volume available
        cluster_4_neuronlist = navis.NeuronList(cluster_4_neurons)

        fig = navis.plot3d(
            cluster_4_neuronlist,
            color=cluster_colors[4],
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 4 Morphology ({len(cluster_4_neurons)} neurons)'
        )

    if fig is not None:
        fig.update_layout(
            scene=dict(
                xaxis_title='X (nm)',
                yaxis_title='Y (nm)',
                zaxis_title='Z (nm)',
                aspectmode='data'
            )
        )
        fig.show()
else:
    print(f'No neurons in cluster 4')


In [23]:
# Visualize Cluster 5
cluster_5_neurons = [neurons[i] for i in range(len(neurons)) if clusters[i] == 5]

if len(cluster_5_neurons) > 0:
    print(f'Cluster 5: {len(cluster_5_neurons)} neurons')

    # Prepare volume if available
    if 'loaded_meshes' in locals() and len(loaded_meshes) > 2:
        leg_mesh = loaded_meshes[2]
        leg_volume = navis.Volume(leg_mesh.vertices, leg_mesh.faces, name='Leg Neuropil', color=(0.827, 0.827, 0.827, 0.3))

        # Create NeuronList and plot with volume
        cluster_5_neuronlist = navis.NeuronList(cluster_5_neurons)

        # CRITICAL: Volumes don't consume colors from plot3d()
        # Only pass neuron colors (navis unpacks NeuronList)
        neuron_colors = [cluster_colors[5]] * len(cluster_5_neuronlist)
        neuron_alphas = [1.0] * len(cluster_5_neuronlist)

        fig = navis.plot3d(
            [leg_volume, cluster_5_neuronlist],
            color=neuron_colors,  # ONLY neuron colors!
            alpha=neuron_alphas,
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 5 Morphology ({len(cluster_5_neurons)} neurons)'
        )
    else:
        # Plot neurons only if no volume available
        cluster_5_neuronlist = navis.NeuronList(cluster_5_neurons)

        fig = navis.plot3d(
            cluster_5_neuronlist,
            color=cluster_colors[5],
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 5 Morphology ({len(cluster_5_neurons)} neurons)'
        )

    if fig is not None:
        fig.update_layout(
            scene=dict(
                xaxis_title='X (nm)',
                yaxis_title='Y (nm)',
                zaxis_title='Z (nm)',
                aspectmode='data'
            )
        )
        fig.show()
else:
    print(f'No neurons in cluster 5')


In [24]:
# Visualize Cluster 6
cluster_6_neurons = [neurons[i] for i in range(len(neurons)) if clusters[i] == 6]

if len(cluster_6_neurons) > 0:
    print(f'Cluster 6: {len(cluster_6_neurons)} neurons')

    # Prepare volume if available
    if 'loaded_meshes' in locals() and len(loaded_meshes) > 2:
        leg_mesh = loaded_meshes[2]
        leg_volume = navis.Volume(leg_mesh.vertices, leg_mesh.faces, name='Leg Neuropil', color=(0.827, 0.827, 0.827, 0.3))

        # Create NeuronList and plot with volume
        cluster_6_neuronlist = navis.NeuronList(cluster_6_neurons)

        # CRITICAL: Volumes don't consume colors from plot3d()
        # Only pass neuron colors (navis unpacks NeuronList)
        neuron_colors = [cluster_colors[6]] * len(cluster_6_neuronlist)
        neuron_alphas = [1.0] * len(cluster_6_neuronlist)

        fig = navis.plot3d(
            [leg_volume, cluster_6_neuronlist],
            color=neuron_colors,  # ONLY neuron colors!
            alpha=neuron_alphas,
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 6 Morphology ({len(cluster_6_neurons)} neurons)'
        )
    else:
        # Plot neurons only if no volume available
        cluster_6_neuronlist = navis.NeuronList(cluster_6_neurons)

        fig = navis.plot3d(
            cluster_6_neuronlist,
            color=cluster_colors[6],
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 6 Morphology ({len(cluster_6_neurons)} neurons)'
        )

    if fig is not None:
        fig.update_layout(
            scene=dict(
                xaxis_title='X (nm)',
                yaxis_title='Y (nm)',
                zaxis_title='Z (nm)',
                aspectmode='data'
            )
        )
        fig.show()
else:
    print(f'No neurons in cluster 6')


In [25]:
# Visualize Cluster 7
cluster_7_neurons = [neurons[i] for i in range(len(neurons)) if clusters[i] == 7]

if len(cluster_7_neurons) > 0:
    print(f'Cluster 7: {len(cluster_7_neurons)} neurons')

    # Prepare volume if available
    if 'loaded_meshes' in locals() and len(loaded_meshes) > 2:
        leg_mesh = loaded_meshes[2]
        leg_volume = navis.Volume(leg_mesh.vertices, leg_mesh.faces, name='Leg Neuropil', color=(0.827, 0.827, 0.827, 0.3))

        # Create NeuronList and plot with volume
        cluster_7_neuronlist = navis.NeuronList(cluster_7_neurons)

        # CRITICAL: Volumes don't consume colors from plot3d()
        # Only pass neuron colors (navis unpacks NeuronList)
        neuron_colors = [cluster_colors[7]] * len(cluster_7_neuronlist)
        neuron_alphas = [1.0] * len(cluster_7_neuronlist)

        fig = navis.plot3d(
            [leg_volume, cluster_7_neuronlist],
            color=neuron_colors,  # ONLY neuron colors!
            alpha=neuron_alphas,
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 7 Morphology ({len(cluster_7_neurons)} neurons)'
        )
    else:
        # Plot neurons only if no volume available
        cluster_7_neuronlist = navis.NeuronList(cluster_7_neurons)

        fig = navis.plot3d(
            cluster_7_neuronlist,
            color=cluster_colors[7],
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 7 Morphology ({len(cluster_7_neurons)} neurons)'
        )

    if fig is not None:
        fig.update_layout(
            scene=dict(
                xaxis_title='X (nm)',
                yaxis_title='Y (nm)',
                zaxis_title='Z (nm)',
                aspectmode='data'
            )
        )
        fig.show()
else:
    print(f'No neurons in cluster 7')


In [26]:
# Visualize Cluster 8
cluster_8_neurons = [neurons[i] for i in range(len(neurons)) if clusters[i] == 8]

if len(cluster_8_neurons) > 0:
    print(f'Cluster 8: {len(cluster_8_neurons)} neurons')

    # Prepare volume if available
    if 'loaded_meshes' in locals() and len(loaded_meshes) > 2:
        leg_mesh = loaded_meshes[2]
        leg_volume = navis.Volume(leg_mesh.vertices, leg_mesh.faces, name='Leg Neuropil', color=(0.827, 0.827, 0.827, 0.3))

        # Create NeuronList and plot with volume
        cluster_8_neuronlist = navis.NeuronList(cluster_8_neurons)

        # CRITICAL: Volumes don't consume colors from plot3d()
        # Only pass neuron colors (navis unpacks NeuronList)
        neuron_colors = [cluster_colors[8]] * len(cluster_8_neuronlist)
        neuron_alphas = [1.0] * len(cluster_8_neuronlist)

        fig = navis.plot3d(
            [leg_volume, cluster_8_neuronlist],
            color=neuron_colors,  # ONLY neuron colors!
            alpha=neuron_alphas,
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 8 Morphology ({len(cluster_8_neurons)} neurons)'
        )
    else:
        # Plot neurons only if no volume available
        cluster_8_neuronlist = navis.NeuronList(cluster_8_neurons)

        fig = navis.plot3d(
            cluster_8_neuronlist,
            color=cluster_colors[8],
            backend='plotly',
            width=1000,
            height=800,
            title=f'Cluster 8 Morphology ({len(cluster_8_neurons)} neurons)'
        )

    if fig is not None:
        fig.update_layout(
            scene=dict(
                xaxis_title='X (nm)',
                yaxis_title='Y (nm)',
                zaxis_title='Z (nm)',
                aspectmode='data'
            )
        )
        fig.show()
else:
    print(f'No neurons in cluster 8')


## Your Turn: New Subset

Now try this analysis yourself with a different dataset!

**Exercise:** Switch the pre-prepared subset to `antennal_lobe`

To work with a different subset, change the subset configuration at the top:
```python
SUBSET_NAME = "antennal_lobe"
NEUROPIL_PATTERN = "AL"
```

Then re-run the entire notebook to see how the results differ!

# Extensions

In the extended code blocks, you can learn how to 1. transform data yourself between template spaces, and 2. visualise data split by axon and dendrite.

---

## Extension 1: Template Brain Transformations

While we provide pre-transformed skeletons, you can also perform transformations yourself using **flybrains** and **navis**.

The [**flybrains**](https://github.com/navis-org/navis-flybrains) package provides template brain transformations for 31+ Drosophila brain spaces including FAFB14, FLYWIRE, JRC2018F/M/U, JRCFIB2022M (maleCNS), FANC, BANC, and MANC.

This is useful when:
- Working with custom neurons not in our dataset
- Transforming synapses or other 3D data
- Needing intermediate template spaces

**Note:** This requires flybrains to be installed. We installed it earlier in this tutorial.

In [27]:
# Check if flybrains is installed
try:
    import flybrains
    print(f"✓ flybrains version: {flybrains.__version__}")
    FLYBRAINS_AVAILABLE = True
except ImportError:
    print("⚠ flybrains not installed")
    print("Install with: pip install flybrains")
    print("Then download transforms (one-time):")
    print("  flybrains.download_jefferislab_transforms()")
    print("  flybrains.download_jrc_transforms()")
    print("  flybrains.download_jrc_vnc_transforms()")
    FLYBRAINS_AVAILABLE = False

if FLYBRAINS_AVAILABLE:
    # Switch to MANC dataset
    dataset_manc = "manc_121"
    dataset_manc_id = "manc_121_id"
    
    # Read MANC meta data
    meta_manc_path = construct_path(DATA_PATH, dataset_manc, "meta")
    meta_manc_full = read_feather_gcs(meta_manc_path, gcs_fs=gcs)
    
    # Filter for DNg12 neurons in MANC
    meta_manc_dng12 = meta_manc_full[
        meta_manc_full['cell_type'].str.contains('DNg12', case=False, na=False)
    ]
    print(f"\nFound {len(meta_manc_dng12)} DNg12 neurons in MANC")
    
    if len(meta_manc_dng12) > 0:
        # Load MANC neurons in BANC space (pre-transformed for convenience)
        # Note: You could also load native MANC space and transform with flybrains
        skeletons_manc_banc_path = construct_path(
            DATA_PATH,
            dataset_manc,
            "skeletons",
            space_suffix="banc_space"
        )
        
        manc_ids = meta_manc_dng12[dataset_manc_id].values[:5]  # Limit to 5 for speed
        manc_filenames = [f"{nid}.swc" for nid in manc_ids]
        
        print(f"Loading {len(manc_ids)} MANC neurons (in BANC space)...")
        
        if USE_GCS:
            gcs_manc_path = skeletons_manc_banc_path.replace("gs://", "")
            manc_dng12_banc = batch_read_swc_from_gcs(
                gcs,
                gcs_manc_path,
                manc_filenames,
                show_progress=False
            )
        else:
            manc_dng12_banc = navis.NeuronList([
                navis.read_swc(f"{skeletons_manc_banc_path}/{fname}")
                for fname in manc_filenames
            ])
        
        print(f"✓ Loaded {len(manc_dng12_banc)} MANC neurons")
        
        # Co-plot BANC DNg12 (navy), maleCNS DNg12 (red), and MANC DNg12 (green)
        # All in BANC space, all in nm coordinates
        if len(neurons) > 0 and len(neurons_male_banc) > 0:
            print("\nCo-plotting neurons from three datasets in BANC space...")
            
            # Limit to 3 neurons per dataset for clarity
            banc_subset = neurons[:3]
            male_subset = neurons_male_banc[:3]
            manc_subset = manc_dng12_banc[:3]
            
            # Combine all neurons
            # Create separate NeuronLists for each dataset
            banc_neuronlist = navis.NeuronList(banc_subset)
            male_neuronlist = navis.NeuronList(male_subset)
            manc_neuronlist = navis.NeuronList(manc_subset)
            
            # CRITICAL: navis unpacks NeuronLists - need one color per neuron
            banc_colors = ['navy'] * len(banc_neuronlist)
            male_colors = ['red'] * len(male_neuronlist)
            manc_colors = ['green'] * len(manc_neuronlist)

            # Plot with navis using list pattern
            fig = navis.plot3d(
                [banc_neuronlist, male_neuronlist, manc_neuronlist],
                color=banc_colors + male_colors + manc_colors,
                backend='plotly',
                width=1400,
                height=900,
                title='DNg12 Neurons: BANC (navy) vs maleCNS (red) vs MANC (green) in BANC Space'
            )
            
            if fig is not None:
                # Update axis labels
                fig.update_layout(
                    scene=dict(
                        xaxis_title='X (nm)',
                        yaxis_title='Y (nm)',
                        zaxis_title='Z (nm)',
                        aspectmode='data'
                    )
                )
                fig.show()
            
            print("\n✓ Cross-dataset visualization complete")
            print("Note: All neurons are in BANC space for direct comparison")
        else:
            print("⚠ Need BANC and maleCNS neurons loaded (from earlier cells)")
    else:
        print("⚠ No DNg12 neurons found in MANC")
else:
    print("\nSkipping transformation example - flybrains not available")

## Extension 2: Axon-Dendrite Splits

For many of our datasets (but notably not BANC yet), we have axon-dendrite splits that have been pre-calculated using a graph-theoretic algorithm from [Schneider-Mizell et al. 2016](https://elifesciences.org/articles/12059).

The method determines axon versus dendrite by "cutting" the neuron at its major information bottleneck based on synapse distributions, observing which half has more inputs versus outputs.

**Data locations:**
- The `Label` column in `.swc` skeleton files gives compartment assignments (values: "axon", "dendrite", "primary_dendrite", "primary_neurite")
- The `pre_label` and `post_label` columns in synapse tables give compartment assignments

Let's visualize some olfactory projection neurons from **FAFB** with compartments color-coded:

- **Cyan** = dendrite
- **Green** = primary dendrite (linker)
- **Orange** = axon  
- **Purple** = primary neurite tract (cell body fiber tract)

We'll plot V_l2PN projections from the V glomerulus, which sense CO2.

In [28]:
# Switch to FAFB dataset (has compartment labels)
dataset_fafb = "fafb_783"
dataset_fafb_id = "fafb_783_id"

# Read FAFB meta data
meta_fafb_path = construct_path(DATA_PATH, dataset_fafb, "meta")
meta_fafb = read_feather_gcs(meta_fafb_path, gcs_fs=gcs)

# Filter for V_l2PN neurons
vpn_meta = meta_fafb[
    meta_fafb['cell_type'].str.contains('V_l2PN', case=False, na=False)
]
print(f"Found {len(vpn_meta)} V_l2PN neurons in FAFB")

if len(vpn_meta) > 0:
    # Get neuron IDs (limit to first 3 for visualization)
    vpn_ids = vpn_meta[dataset_fafb_id].values[:min(3, len(vpn_meta))]

    # Construct path to FAFB skeletons in BANC space (for co-visualization)
    skeletons_fafb_path = construct_path(
        DATA_PATH,
        "fafb",
        "skeletons",
        space_suffix="banc_space"
    )

    print(f"Loading {len(vpn_ids)} FAFB V_l2PN neurons...")

    # Read FAFB neurons
    vpn_filenames = [f"{nid}.swc" for nid in vpn_ids]

    if USE_GCS:
        gcs_fafb_path = skeletons_fafb_path.replace("gs://", "")
        vpn_neurons = batch_read_swc_from_gcs(
            gcs,
            gcs_fafb_path,
            vpn_filenames,
            show_progress=False
        )
    else:
        vpn_neurons = navis.NeuronList([
            navis.read_swc(f"{skeletons_fafb_path}/{fname}")
            for fname in vpn_filenames
        ])

    print(f"✓ Loaded {len(vpn_neurons)} V_l2PN neurons")

    if len(vpn_neurons) > 0:
        # Split neurons by compartment (similar to R's hemibrainr functions)
        # This uses the 'Label' column from SWC files
        compartments = split_neurons_by_compartment(vpn_neurons)

        axon_list = compartments['axon']
        dendrite_list = compartments['dendrite']
        linker_list = compartments['linker']
        neurite_list = compartments['neurite']

        print(f"\nCompartments found:")
        print(f"  Axons: {len(axon_list)}")
        print(f"  Dendrites: {len(dendrite_list)}")
        print(f"  Linkers (primary dendrite): {len(linker_list)}")
        print(f"  Neurites (primary neurite): {len(neurite_list)}")

        # Try to load antennal lobe mesh for context
        try:
            al_mesh_path = construct_path(
                DATA_PATH,
                "fafb",
                "neuropil_meshes",
                space_suffix="banc_space"
            )

            al_mesh_file = "AL_R.obj"

            if USE_GCS:
                gcs_al_path = al_mesh_path.replace("gs://", "") + "/" + al_mesh_file
                al_mesh = read_obj_from_gcs(gcs, gcs_al_path)
            else:
                al_mesh = trimesh.load_mesh(f"{al_mesh_path}/{al_mesh_file}")

            # Convert trimesh to navis Volume
            al_volume = navis.Volume(
                vertices=al_mesh.vertices,
                faces=al_mesh.faces,
                name='Antennal Lobe',
                color=(0.8, 0.8, 0.8, 0.1)  # Light grey, transparent
            )
            print("✓ Loaded antennal lobe mesh")
        except Exception as e:
            print(f"  ⚠ Could not load antennal lobe mesh: {e}")
            al_volume = None

        # Plot compartments with different colors (similar to R code)
        # This approach matches hemibrainr::axonic_cable(), dendritic_cable(), etc.
        print("\nVisualizing V_l2PN neurons by compartment...")

        # Build plot objects and colors manually
        plot_objects = []
        plot_colors = []

        # Add antennal lobe volume if available
        if al_volume is not None:
            plot_objects.append(al_volume)
            # Volume color is set on the object itself

        # Add compartments in order (matching R code)
        # dendrite = cyan
        if len(dendrite_list) > 0:
            for n in dendrite_list:
                plot_objects.append(n)
                plot_colors.append('cyan')

        # linker (primary dendrite) = green
        if len(linker_list) > 0:
            for n in linker_list:
                plot_objects.append(n)
                plot_colors.append('green')

        # axon = orange
        if len(axon_list) > 0:
            for n in axon_list:
                plot_objects.append(n)
                plot_colors.append('orange')

        # neurite (primary neurite) = purple
        if len(neurite_list) > 0:
            for n in neurite_list:
                plot_objects.append(n)
                plot_colors.append('purple')

        # Plot with navis.plot3d
        if len(plot_objects) > 0:
            # Adjust colors list to account for volume (if present)
            if al_volume is not None:
                # Volume doesn't consume a color from the list
                # navis.plot3d will use the color set on the Volume object
                fig = navis.plot3d(
                    plot_objects,
                    color=['lightgrey'] + plot_colors,  # First color for volume, rest for neurons
                    backend='plotly',
                    width=1200,
                    height=800,
                    title='FAFB V_l2PN Neurons - Colored by Compartment (in Antennal Lobe)'
                )
            else:
                fig = navis.plot3d(
                    plot_objects,
                    color=plot_colors,
                    backend='plotly',
                    width=1200,
                    height=800,
                    title='FAFB V_l2PN Neurons - Colored by Compartment'
                )

            if fig is not None:
                fig.show()

            print("\n✓ Compartment visualization complete")
        else:
            print("\n⚠ No compartments to plot")
    else:
        print("⚠ No V_l2PN neurons loaded")
else:
    print("⚠ No V_l2PN neurons found in FAFB - skipping compartment visualization")


### Visualize Synapses by Compartment

We can also visualize the synapses on these neurons, colored by compartment and direction:

- **Navy** = input synapse, dendrite
- **Cyan** = input synapse, axon
- **Pink** = output synapse, dendrite
- **Red** = output synapse, axon

In [29]:
# Only run if vpn_ids exists (from previous section)
if 'vpn_ids' in locals() and len(vpn_ids) > 0:
    # Construct path to FAFB synapses file
    dataset = "fafb_783"
    synapses_path = f"{DATA_PATH}/fafb/antennal_lobe/{dataset}_antennal_lobe_synapses.feather"
    
    print("Loading FAFB antennal lobe synapses...")
    
    # Read synapses and filter for V_l2PN neurons
    vpn_synapses = read_feather_gcs(synapses_path, gcs_fs=gcs)
    vpn_synapses = vpn_synapses[
        vpn_synapses['pre'].isin(vpn_ids) | vpn_synapses['post'].isin(vpn_ids)
    ]
    
    print(f"Found {len(vpn_synapses):,} synapses for V_l2PN neurons")
    
    if len(vpn_synapses) > 0:
        # Create synapse classifications
        # Determine if input (post) or output (pre)
        # Determine compartment label (axon vs dendrite)
        
        # Output synapses on axons
        vpn_synapses_output_axons = vpn_synapses[
            vpn_synapses['pre'].isin(vpn_ids) & (vpn_synapses['pre_label'] == 'axon')
        ][['x', 'y', 'z']].values
        
        # Output synapses on dendrites
        vpn_synapses_output_dendrites = vpn_synapses[
            vpn_synapses['pre'].isin(vpn_ids) & (vpn_synapses['pre_label'] == 'dendrite')
        ][['x', 'y', 'z']].values
        
        # Input synapses on axons
        vpn_synapses_input_axons = vpn_synapses[
            vpn_synapses['post'].isin(vpn_ids) & (vpn_synapses['post_label'] == 'axon')
        ][['x', 'y', 'z']].values
        
        # Input synapses on dendrites
        vpn_synapses_input_dendrites = vpn_synapses[
            vpn_synapses['post'].isin(vpn_ids) & (vpn_synapses['post_label'] == 'dendrite')
        ][['x', 'y', 'z']].values
        
        print(f"\nSynapse counts by type:")
        print(f"  Output, axon: {len(vpn_synapses_output_axons):,}")
        print(f"  Output, dendrite: {len(vpn_synapses_output_dendrites):,}")
        print(f"  Input, axon: {len(vpn_synapses_input_axons):,}")
        print(f"  Input, dendrite: {len(vpn_synapses_input_dendrites):,}")
        
        # Plot neurons with synapses using navis.plot3d and manual plotly additions
        # Start with neuron plot
        if 'vpn_neurons' in locals() and len(vpn_neurons) > 0:
            print("\nVisualizing synapses by compartment...")
            
            # Plot neurons first
            fig = navis.plot3d(
                vpn_neurons,
                color='lightgrey',
                backend='plotly',
                width=1400,
                height=900,
                title='V_l2PN Synapses by Compartment and Direction'
            )
            
            if fig is not None:
                # Add synapse points as scatter3d traces
                # Navy = input synapse, dendrite
                if len(vpn_synapses_input_dendrites) > 0:
                    fig.add_trace(go.Scatter3d(
                        x=vpn_synapses_input_dendrites[:, 0],
                        y=vpn_synapses_input_dendrites[:, 1],
                        z=vpn_synapses_input_dendrites[:, 2],
                        mode='markers',
                        marker=dict(size=2, color='navy', opacity=0.6),
                        name='Input, dendrite',
                        showlegend=True
                    ))
                
                # Cyan = input synapse, axon
                if len(vpn_synapses_input_axons) > 0:
                    fig.add_trace(go.Scatter3d(
                        x=vpn_synapses_input_axons[:, 0],
                        y=vpn_synapses_input_axons[:, 1],
                        z=vpn_synapses_input_axons[:, 2],
                        mode='markers',
                        marker=dict(size=2, color='cyan', opacity=0.6),
                        name='Input, axon',
                        showlegend=True
                    ))
                
                # Pink = output synapse, dendrite
                if len(vpn_synapses_output_dendrites) > 0:
                    fig.add_trace(go.Scatter3d(
                        x=vpn_synapses_output_dendrites[:, 0],
                        y=vpn_synapses_output_dendrites[:, 1],
                        z=vpn_synapses_output_dendrites[:, 2],
                        mode='markers',
                        marker=dict(size=2, color='pink', opacity=0.6),
                        name='Output, dendrite',
                        showlegend=True
                    ))
                
                # Red = output synapse, axon
                if len(vpn_synapses_output_axons) > 0:
                    fig.add_trace(go.Scatter3d(
                        x=vpn_synapses_output_axons[:, 0],
                        y=vpn_synapses_output_axons[:, 1],
                        z=vpn_synapses_output_axons[:, 2],
                        mode='markers',
                        marker=dict(size=2, color='red', opacity=0.6),
                        name='Output, axon',
                        showlegend=True
                    ))
                
                # Update layout
                fig.update_layout(
                    scene=dict(
                        xaxis_title='X (nm)',
                        yaxis_title='Y (nm)',
                        zaxis_title='Z (nm)',
                        aspectmode='data'
                    )
                )
                
                fig.show()
                print("\n✓ Synapse visualization complete")
        else:
            print("⚠ Neurons not available for plotting")
    else:
        print("⚠ No synapses found for these neurons")
else:
    print("⚠ Skipping synapse visualization - neurons not loaded from previous section")

## Extension 3: Plotting Neuronal Meshes from BANC

So far, we've worked with pre-prepared skeleton data (SWC files) and meshes (OBJ files) downloaded from Google Cloud Storage. However, you can also read mesh data **directly** from the major connectome projects using Python packages like [`caveclient`](https://caveclient.readthedocs.io/) and [`meshparty`](https://meshparty.readthedocs.io/).

This is particularly useful for BANC, where meshes aren't pre-packaged for the course but can be downloaded on-demand from the CAVE (Connectome Annotation Versioning Engine) infrastructure.

### What You'll Need

1. **Python packages:** `caveclient` and `meshparty`
   ```bash
   pip install caveclient meshparty
   ```

2. **BANC Authentication (optional):** While some data is publicly accessible, you may need a BANC API token for full access:
   - Get a token from: https://global.daf-apis.com/auth/api/v1/create_token
   - Save to `~/.cloudvolume/secrets/cave-secret.json`:
     ```json
     {"token": "YOUR_TOKEN_HERE"}
     ```

3. **Patience:** Mesh downloads can take time (especially for large neurons), but they're cached locally for reuse.

### What We'll Do

In this extension, we'll:
- Connect to the BANC datastack using `caveclient`
- Download meshes for 5 EPG neurons (from the ellipsoid body)
- Convert them to `navis.MeshNeuron` objects
- Visualize them with `navis.plot3d()` in rainbow colours

**Note:** The neuroglancer scene at https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/5181712630808576 shows these neurons in context, but we'll download and visualize the actual mesh data using the API.


In [30]:
# Check if caveclient and meshparty are available
try:
    import caveclient
    from meshparty import trimesh_io
    CAVE_AVAILABLE = True
    print(f"✓ caveclient version: {caveclient.__version__}")
    print("✓ meshparty available")
except ImportError as e:
    CAVE_AVAILABLE = False
    print(f"✗ Required packages not installed: {e}")
    print("  Install with: pip install caveclient meshparty")
    print("  Skipping Extension 03...")

if CAVE_AVAILABLE:
    print()
    print("="*70)
    print("Extension 03: Downloading and Visualizing BANC Meshes")
    print("="*70)

    # Step 1: Get EPG neuron IDs from metadata
    print("\n[1/4] Finding EPG neurons in metadata...")
    epg_mask = meta_full['cell_type'].str.contains('EPG', case=False, na=False)
    epg_meta = meta_full[epg_mask]
    print(f"✓ Found {len(epg_meta)} EPG neurons")

    # Use first 5 EPG neurons for visualization
    epg_ids = epg_meta[DATASET_ID].values[:5]
    print(f"  Will download {len(epg_ids)} meshes")

    # Step 2: Connect to BANC
    print("\n[2/4] Connecting to BANC...")
    try:
        client = caveclient.CAVEclient('brain_and_nerve_cord')
        print(f"✓ Connected to {client.datastack_name}")
    except Exception as e:
        print(f"✗ Failed to connect: {e}")
        print("  You may need to set up authentication:")
        print("  Get a token from: https://global.daf-apis.com/auth/api/v1/create_token")
        print("  Save to ~/.cloudvolume/secrets/cave-secret.json")
        CAVE_AVAILABLE = False

    if CAVE_AVAILABLE:
        # Step 3: Set up mesh downloading with cache
        print("\n[3/4] Setting up mesh cache...")
        mesh_cache_dir = os.path.expanduser("~/.banc_meshes")
        os.makedirs(mesh_cache_dir, exist_ok=True)
        print(f"✓ Cache directory: {mesh_cache_dir}")

        mm = trimesh_io.MeshMeta(
            cv_path=client.info.segmentation_source(),
            disk_cache_path=mesh_cache_dir
        )
        print("✓ MeshMeta initialized for mesh downloading")

        # Step 4: Download meshes
        print(f"\n[4/4] Downloading {len(epg_ids)} EPG neuron meshes...")
        print("  (This may take a minute...)")

        banc_meshes = []
        for i, seg_id in enumerate(epg_ids, 1):
            try:
                print(f"  [{i}/{len(epg_ids)}] Segment {seg_id}...", end=" ")

                # Download mesh (meshparty returns trimesh.Trimesh)
                mesh = mm.mesh(seg_id=int(seg_id))

                # Convert to navis MeshNeuron
                mesh_neuron = navis.MeshNeuron(
                    mesh,
                    id=seg_id,
                    name=f"EPG_{seg_id}"
                )
                banc_meshes.append(mesh_neuron)
                print(f"✓ ({mesh_neuron.n_vertices:,} vertices)")

            except Exception as e:
                print(f"✗ {e}")
                continue

        print(f"\n✓ Successfully downloaded {len(banc_meshes)}/{len(epg_ids)} meshes")

        # Visualize with navis.plot3d()
        if len(banc_meshes) > 0:
            print("\nCreating visualization...")

            # Generate rainbow colors for each mesh
            colors = plt.cm.rainbow(np.linspace(0, 1, len(banc_meshes)))
            color_list = [tuple(c[:3]) for c in colors]

            # Plot with navis
            fig = navis.plot3d(
                banc_meshes,
                color=color_list,
                backend='plotly',
                width=1000,
                height=800,
                title=f'{len(banc_meshes)} EPG Neuron Meshes from BANC'
            )

            if fig is not None:
                fig.update_layout(
                    scene=dict(
                        xaxis_title='X (nm)',
                        yaxis_title='Y (nm)',
                        zaxis_title='Z (nm)',
                        aspectmode='data'
                    )
                )
                fig.show()
                print("✓ Visualization complete!")
            else:
                print("✗ Failed to create visualization")
        else:
            print("\n✗ No meshes downloaded - cannot create visualization")


In [ ]:
# One-time setup: Download transformation registrations
# Uncomment and run once:
import flybrains
flybrains.download_jefferislab_transforms()
flybrains.download_jrc_transforms()
flybrains.download_jrc_vnc_transforms()
flybrains.register_transforms()

# Switch to MANC dataset
dataset_manc = "manc_121"
dataset_manc_id = "manc_121_id"

# Read MANC meta data
meta_manc_path = construct_path(DATA_PATH, dataset_manc, "meta")
meta_manc_full = read_feather_gcs(meta_manc_path, gcs_fs=gcs)

# Filter for DNg12 neurons in MANC
meta_manc_dng12 = meta_manc_full[
    meta_manc_full['cell_type'].str.contains('DNg12', case=False, na=False)
]
print(f"Found {len(meta_manc_dng12)} DNg12 neurons in MANC")

# Construct path to MANC skeletons in NATIVE MANC space
skeletons_manc_path = construct_path(
    DATA_PATH,
    dataset_manc,
    "skeletons",
    space_suffix=f"{dataset_manc}_manc_space"
)

# Read MANC neurons in native space
manc_ids = meta_manc_dng12[dataset_manc_id].values
manc_filenames = [f"{nid}.swc" for nid in manc_ids]

if USE_GCS:
    gcs_manc_path = skeletons_manc_path.replace("gs://", "")
    manc_dng12 = batch_read_swc_from_gcs(
        gcs,
        gcs_manc_path,
        manc_filenames,
        show_progress=False
    )
else:
    manc_dng12 = navis.NeuronList([
        navis.read_swc(f"{skeletons_manc_path}/{fname}")
        for fname in manc_filenames
    ])

print(f"Loaded {len(manc_dng12)} MANC neurons in native space")

# Transform MANC neurons to JRCVNC2018F template
manc_dng12_jrcvnc2018f = navis.xform_brain(
    manc_dng12,
    source='MANC',
    target='JRCVNC2018F'
)

# Transform from JRCVNC2018F to BANC
# Note: For this step, you would need bancr-equivalent functionality in Python
# This is currently only available in R via bancr::banc_to_JRC2018F()
# For now, use the pre-transformed skeletons in BANC space
print("\nNote: Direct JRCVNC2018F -> BANC transform requires additional setup.")
print("Alternative: Use pre-transformed skeletons with space_suffix='banc_space'")

# Visualize the intermediate transform
manc_dng12_jrcvnc2018f_um = manc_dng12_jrcvnc2018f / 1000

fig = navis.plot3d(
    manc_dng12_jrcvnc2018f_um,
    backend='plotly',
    color='green',
    width=1200,
    height=800,
    title="MANC DNg12 Neurons in JRCVNC2018F Space"
)

if fig is not None:
    fig.show()
else:
    print("Note: Plot was displayed inline or could not be generated as a figure object")

print("\nTransformation notes:")
print("- MANC -> JRCVNC2018F: Available via flybrains")
print("- JRCVNC2018F -> BANC: Requires additional bridging registration")
print("- For production use: Use our pre-transformed skeletons in BANC space")


---

## Summary

In this tutorial, we covered:

1. **Loading Neuron Skeletons**: Reading SWC files from GCS with navis
2. **3D Visualization**: Interactive plotting with plotly backend
3. **Morphometric Analysis**: Extracting cable length, branch counts, etc.
4. **Neuropil Meshes**: Visualizing neurons in anatomical context
5. **NBLAST**: Morphological similarity comparison
6. **Template Brains**: Introduction to flybrains for spatial transformations

### Key navis Features

- **Simple API**: `navis.read_swc()`, `navis.plot3d()`, `navis.nblast()`
- **Fast**: Compiled code (Rust) for NBLAST and other operations
- **Flexible**: Multiple backends (plotly, octarine, k3d)
- **Integrated**: Works with pandas, numpy, and other scientific Python tools
- **Well-documented**: Extensive [documentation](https://navis.readthedocs.io/) and [tutorials](https://navis-org.github.io/navis/)

### Useful navis Functions

```python
# Reading neurons
navis.read_swc('neuron.swc')
navis.read_swc('neurons.zip', include_subdirs=True)

# Morphometrics
n.cable_length  # Total cable length
n.n_branches    # Number of branches
n.n_nodes       # Number of nodes
navis.segment_analysis(n)  # Detailed per-segment metrics

# Visualization
navis.plot3d(neurons, backend='plotly', color='blue')
navis.plot3d(neurons, backend='octarine')  # Modern, fast backend

# Morphological comparison
dotprops = navis.make_dotprops(neurons, k=5)
scores = navis.nblast(dotprops, dotprops)

# Spatial transformations
navis.xform_brain(neuron, source='BANC', target='JRC2018F')

# Manipulation
neurons / 1000  # Scale (e.g., nm to µm)
navis.downsample_neuron(n, factor=5)  # Reduce resolution
navis.prune_by_strahler(n, to_prune=1)  # Prune terminal branches
```

---

## Next Steps

- Explore Tutorial 03 for connectivity analysis
- Try different datasets (FAFB, MANC, etc.)
- Experiment with different NBLAST parameters
- Use flybrains to compare neurons across datasets
- Check out the [navis documentation](https://navis.readthedocs.io/) for more advanced features

---

**Tutorial complete!** 🎉

## Session Information

In [31]:
import sys
import platform

print("Python Session Information")
print("=" * 50)
print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")
print("\nPackage Versions:")
print(f"  navis: {navis.__version__}")
print(f"  pandas: {pd.__version__}")
print(f"  numpy: {np.__version__}")

try:
    import flybrains
    print(f"  flybrains: {flybrains.__version__}")
except:
    print(f"  flybrains: not installed")

print("\n" + "=" * 50)

Python Session Information
Python version: 3.10.19 (main, Oct 21 2025, 16:37:10) [Clang 20.1.8 ]
Platform: macOS-15.6.1-arm64-arm-64bit

Package Versions:
  navis: 1.10.0
  pandas: 2.3.3
  numpy: 2.2.6
  flybrains: 0.6.3

